In [ ]:
!pip install xformers

In [ ]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .

In [ ]:
%cd /content/diffusers/examples/text_to_image
!pip install -r requirements.txt

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config()

In [ ]:
from huggingface_hub import HfFolder
HfFolder.save_token('TOKEN GOES HERE')

In [ ]:
MODEL_NAME="stabilityai/stable-diffusion-2"
dataset_name="lmoroney/misato"

!accelerate launch --mixed_precision="fp16"  train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$dataset_name \
  --image_column="image" \
  --caption_column="prompt" \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --max_train_steps=50000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --enable_xformers_memory_efficient_attention \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="/content/lm-misato-model" \
  --checkpointing_steps=2500

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_path = "/content/lm-misato-model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="a detailed photograph of <lora-misato-token> wearing a tiger outfit").images[0]
image.save("/content/yoda-naruto.png")

In [ ]:
!zip -r /content/my_directory.zip /content/lm-misato-lora

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2" \
  --dataset_name="lmoroney/misato" \
  --caption_column="prompt" \
  --resolution=512 \
  --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=1000 \
  --checkpointing_steps=5000 \
  --learning_rate=1e-04 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --seed=42 \
  --output_dir="/content/lm-misato-lora" \
  --validation_prompt="a picture of <lm-misato-lora> on the moon"

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2",
    torch_dtype=torch.float16
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("/content/lm-misato-lora")
pipe.to("cuda")

prompt = "fashion editorial, a portrait of <lm-misato-lora> in water drops, wet hair and skeen, mystical gaze, highres, realistic photo, professional photography, cinematic angle, dynamic light back shining,"
prompt = "black and white photography in style by Bruce Davidson, fashion editorial, <lm-misato-lora> of long exposure photography through artistic lens,  highres, realistic photo, professional photography, cinematic angle, dynamic light back shining, bokeh,"
image = pipe(prompt, num_inference_steps=30).images[0]
image.save("/content/output.png")

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
model_id = "stabilityai/stable-diffusion-2"

device = "cuda" if torch.cuda.is_available() else "cpu"
# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(
    model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe.load_lora_weights("/content/lm-misato-lora")
pipe = pipe.to(device)

prompt = "fashion editorial, a portrait of <lm-misato-lora> in studio red and blue lights in style Alain Laboile, shiny eyes, highres, realistic photo, professional photography, cinematic angle, dynamic light back shining,"
negative_prompt = "(deformed, distorted, disfigured:1.3) , poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4) , disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation"
steps = 50
scale = 9
num_images_per_prompt = 1
seed = 42
generator = torch.Generator(device=device).manual_seed(seed)
image = pipe(prompt, negative_prompt=negative_prompt, width=768, height=1344, num_inference_steps=steps,
             guidance_scale=scale, num_images_per_prompt=num_images_per_prompt, generator=generator).images[0]

image.save("/content/lora-with-negative.png")